In [55]:
import cv2 
import os
import csv
import numpy as np 
# del Zeros
# Zeros = np.zeros((10, 10))
from cv2 import aruco
from torchvision import transforms
from torchvision.models import efficientnet_v2_s  
import math
import networkx as nx
import torch
import torch.nn as nn
import super_resolution
from pathlib import Path
import time
import tensorflow as tf
from tensorflow.keras.preprocessing import image as tf_image
from tensorflow.keras.applications.efficientnet import preprocess_input
tf.config.run_functions_eagerly(True)


In [56]:
def detect_ArUco_details(image): 
    ArUco_details_dict = {}
    ArUco_corners = {}
    
    ##############	ADD YOUR CODE HERE	##############
    aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
    arucoParams = aruco.DetectorParameters()
    # GrayScale Conversion
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect ArUco markers
    corners, ids, _ = aruco.detectMarkers(gray_image, aruco_dict, parameters=arucoParams)

    if ids is not None:
        for i in range(len(ids)):
            marker_id = int(ids[i][0])
            marker_center = [int(coord) for coord in list(np.mean(corners[i][0], axis=0).astype(int))]

            # Store details in dictionaries
            ArUco_details_dict[marker_id] = [marker_center, 0]
            ArUco_corners[marker_id] = [[int(corner[0]), int(corner[1])] for corner in corners[i][0]]
    ##################################################
    
    return ArUco_details_dict, ArUco_corners 

In [57]:
def task_4a_return(image_path, threshold):
    # global cap
    identified_labels = {}  
    
    # ret, frame = cap.read()
    # display_frame = cv2.resize(frame, (960, 540))

    # # Create a named window
    # cv2.namedWindow("Live Feed", cv2.WINDOW_NORMAL)

    # # Flag to check if the picture has been taken
    # picture_taken = False
    # # Get start time
    # start_time = time.time()
    # # Read and display frames from the camera

    # while not picture_taken:
    #     ret, frame = cap.read()
    #     display_frame = cv2.resize(frame, (960, 540))

    #     if not ret:
    #         print("Error reading frame from the camera")
    #         break

    #     cv2.imshow("Live Feed", display_frame)

    #     # Move the window to the left
    #     cv2.moveWindow("Live Feed", 0, 0)

    #     if time.time() - start_time >= 5:
    #         cv2.imwrite(image_path, frame, [cv2.IMWRITE_JPEG_QUALITY, 100])
    #         picture_taken = True

    #     if cv2.waitKey(1) & 0xFF == ord('q'):
    #         break


    # cv2.destroyAllWindows()

    
    img = cv2.imread(image_path)
    # os.remove(image_path)
    # cv2.imshow("Marked", img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    _, corners = detect_ArUco_details(img)
    # mark = mark_ArUco_image(img,  details, corners)

    marking_img = np.copy(img)
    _, corners = detect_ArUco_details(marking_img)
    
    events = [
        [[corners[25][3][0], corners[21][0][1]], [corners[21][0][0], corners[7][1][1]-10]],
        [[corners[31][1][0], corners[28][1][1]], [corners[30][0][0], corners[14][3][1]]],
        [corners[31][1], [corners[30][0][0], corners[11][3][1]]], 
        [[corners[25][0][0], corners[34][0][1]], [corners[34][0][0], corners[11][3][1]]], 
        [[corners[42][1][0], corners[52][1][1]], [corners[40][0][0], corners[10][3][1]-30]]   
    ]

    i=1
    eventlist=[]
    letters = {1: "A", 2: "B", 3: "C", 4: "D", 5: "E"}
    classconv = { "combat": "Combat", "destroyedbuilding": "Destroyed buildings", 
                 "humanitarianaid": "Humanitarian Aid and rehabilitation",
                 "militaryvehicles": "Military Vehicles", "fire": "Fire", "blank": "Blank"}

    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = efficientnet_v2_s().to(device)
    
    model.classifier = torch.nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features=1280, out_features=5, bias=True),
    ).to(device)


    # model.classifier = torch.nn.Sequential(
    #     nn.Linear(1280, 256), # Additional linear layer with 256 output features
    #     # nn.ReLU(inplace=True),         # Activation function
    #     nn.Dropout(p=0.5, inplace=True),
    #     nn.Linear(256, 5)
    # ).to(device)

    # model.load_state_dict(torch.load('weights/gajjar.tf'))
    # model.load_state_dict(torch.load('weights/w1.tf'))
    model = torch.load('weights/model.pth')


    model.eval()
    
    image_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224, 224), antialias=False),
            
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    temp = 'output/temp.jpg'

    for i, (tl, br) in enumerate(events):
        tl_adj = [tl[0] + 10, tl[1] + 7]
        br_adj = [br[0] - 10, br[1] - 4]
        roi = img[tl_adj[1]:br_adj[1], tl_adj[0]:br_adj[0]]

        reference = cv2.imread(f"images/empty/{i}.jpg")
        
        # Perform morphological opening
        kernel = np.ones((5,5),np.uint8)
        opened = cv2.morphologyEx(roi, cv2.MORPH_OPEN, kernel)

        gray = cv2.cvtColor(opened, cv2.COLOR_BGR2GRAY)

        # Apply adaptive thresholding to the image
        binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

        # Find contours in the image
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Sort contours by area in descending order, take the first one (the largest)
        contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

        # Get the bounding rectangle of the largest contour
        x, y, w, h = cv2.boundingRect(contour)

        # Crop the image using the bounding rectangle, add some padding if needed
        padding = 0  # adjust this value according to your needs
        crop = roi[max(0, y-padding):min(y+h+padding, roi.shape[0]), max(0, x-padding):min(x+w+padding, roi.shape[1])]

        # Convert the cropped image to HSV color space
        # hsv_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)

        # img_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        # lower_green = np.array([0, 50, 0])
        # upper_green = np.array([50, 100, 50])

        img_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        lower_green = np.array([200, 200, 200])
        upper_green = np.array([255, 255, 255])

        # Threshold the RGB image to get only green colors
        mask = cv2.inRange(img_rgb, lower_green, upper_green)

        # Calculate the percentage of green pixels
        green_percentage = (np.count_nonzero(mask) / (crop.shape[0] * crop.shape[1])) * 100
        # print(green_percentage)

        # If the majority of the cropped area is green, skip the classification
        threshold_percentage = 80  # Adjust the threshold as needed
        cv2.imwrite(temp, crop, [cv2.IMWRITE_JPEG_QUALITY, 100])
        crop = cv2.imread(temp)

        eventlist.append(crop)
        cv2.imwrite(temp, crop, [cv2.IMWRITE_JPEG_QUALITY, 100])
        result = cv2.imread(temp, cv2.IMREAD_COLOR)
        result = super_resolution.cartoon_upsampling_4x(temp, temp )


        with torch.inference_mode():
            # 6. Transform and add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
            transformed_image = image_transform(result).unsqueeze(dim=0)
            # 7. Make a prediction on image with an extra dimension and send it to the target device
            target_image_pred = model(transformed_image.to(device))

        # 8. Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
        target_image_pred_probs = torch.softmax(target_image_pred, dim=1)
        
        # 9. Convert prediction probabilities -> prediction labels
        pred = torch.argmax(target_image_pred_probs, dim=1)
        class_names = ['combat', 'destroyedbuilding', 'fire', 'humanitarianaid', 'militaryvehicles']

        print(target_image_pred_probs[0])

        if max(target_image_pred_probs[0]) < threshold[i]:
            event = "blank"
        else:
            
            event = class_names[pred]

        offset_x = tl_adj[0] + x - 10
        offset_y = tl_adj[1] + y - 10    

        box = cv2.rectangle(marking_img, (offset_x, offset_y), (offset_x + w + 20, offset_y + h + 20), (0, 255, 0), 2)
        
        offset_y -= 10
        font = cv2.FONT_HERSHEY_SIMPLEX
        scale = 0.7
        thickness = 1
        text = classconv[event]
        (text_width, text_height), _ = cv2.getTextSize(text, font, scale, thickness)

        cv2.rectangle(marking_img, (offset_x, offset_y - text_height - 10), (offset_x + text_width, offset_y), (140, 133, 133), -1)
        cv2.putText(box, text, (offset_x, offset_y - 10), cv2.FONT_HERSHEY_SIMPLEX, scale, (0,255,0), thickness)
        identified_labels[letters[i+1]] = classconv[event]
        # display_frame = cv2.resize(marking_img, (960, 540))
        cv2.imshow("Marked Image", marking_img)
        
        cv2.waitKey(500)  # delay for 500 milliseconds
        
    # display_frame = 
    cv2.imshow("Marked Image", marking_img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return identified_labels



In [58]:
def task_4a_returnkeras(image_path, threshold):
    # ... (the rest of the function stays the same until the model loading part)
        # global cap
    identified_labels = {}  
    
    # ret, frame = cap.read()
    # display_frame = cv2.resize(frame, (960, 540))

    # # Create a named window
    # cv2.namedWindow("Live Feed", cv2.WINDOW_NORMAL)

    # # Flag to check if the picture has been taken
    # picture_taken = False
    # # Get start time
    # start_time = time.time()
    # # Read and display frames from the camera

    # while not picture_taken:
    #     ret, frame = cap.read()
    #     display_frame = cv2.resize(frame, (960, 540))

    #     if not ret:
    #         print("Error reading frame from the camera")
    #         break

    #     cv2.imshow("Live Feed", display_frame)

    #     # Move the window to the left
    #     cv2.moveWindow("Live Feed", 0, 0)

    #     if time.time() - start_time >= 5:
    #         cv2.imwrite(image_path, frame, [cv2.IMWRITE_JPEG_QUALITY, 100])
    #         picture_taken = True

    #     if cv2.waitKey(1) & 0xFF == ord('q'):
    #         break


    # cv2.destroyAllWindows()

    
    img = cv2.imread(image_path)
    # os.remove(image_path)
    # cv2.imshow("Marked", img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    _, corners = detect_ArUco_details(img)
    # mark = mark_ArUco_image(img,  details, corners)

    marking_img = np.copy(img)
    _, corners = detect_ArUco_details(marking_img)
    
    events = [
        [[corners[25][3][0], corners[21][0][1]], [corners[21][0][0], corners[7][1][1]-10]],
        [[corners[31][1][0], corners[28][1][1]], [corners[30][0][0], corners[14][3][1]]],
        [corners[31][1], [corners[30][0][0], corners[11][3][1]]], 
        [[corners[25][0][0], corners[34][0][1]], [corners[34][0][0], corners[11][3][1]]], 
        [[corners[42][1][0], corners[52][1][1]], [corners[40][0][0], corners[10][3][1]-30]]   
    ]

    i=1
    eventlist=[]
    letters = {1: "A", 2: "B", 3: "C", 4: "D", 5: "E"}
    classconv = { "combat": "Combat", "destroyedbuilding": "Destroyed buildings", 
                 "humanitarianaid": "Humanitarian Aid and rehabilitation",
                 "militaryvehicles": "Military Vehicles", "fire": "Fire", "blank": "Blank"}

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load the TensorFlow model
    model = tf.keras.models.load_model('models/bruh99.h5', compile=False)
    # IMG_SIZE = (224, 224)


    # base_model = tf.keras.applications.EfficientNetV2S(input_shape=IMG_SHAPE,
    #                                            include_top=False,
    #                                            weights='imagenet')

    # ... (the rest of the function stays the same until the image preprocessing part)
    
    # ret, frame = cap.read()
    # display_frame = cv2.resize(frame, (960, 540))

    # # Create a named window
    # cv2.namedWindow("Live Feed", cv2.WINDOW_NORMAL)

    # # Flag to check if the picture has been taken
    # picture_taken = False
    # # Get start time
    # start_time = time.time()
    # # Read and display frames from the camera

    # while not picture_taken:
    #     ret, frame = cap.read()
    #     display_frame = cv2.resize(frame, (960, 540))

    #     if not ret:
    #         print("Error reading frame from the camera")
    #         break

    #     cv2.imshow("Live Feed", display_frame)

    #     # Move the window to the left
    #     cv2.moveWindow("Live Feed", 0, 0)

    #     if time.time() - start_time >= 5:
    #         cv2.imwrite(image_path, frame, [cv2.IMWRITE_JPEG_QUALITY, 100])
    #         picture_taken = True

    #     if cv2.waitKey(1) & 0xFF == ord('q'):
    #         break


    # cv2.destroyAllWindows()

    
    img = cv2.imread(image_path)
    # os.remove(image_path)
    # cv2.imshow("Marked", img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    _, corners = detect_ArUco_details(img)
    # mark = mark_ArUco_image(img,  details, corners)

    marking_img = np.copy(img)
    _, corners = detect_ArUco_details(marking_img)
    
    events = [
        [[corners[25][3][0], corners[21][0][1]], [corners[21][0][0], corners[7][1][1]-10]],
        [[corners[31][1][0], corners[28][1][1]], [corners[30][0][0], corners[14][3][1]]],
        [corners[31][1], [corners[30][0][0], corners[11][3][1]]], 
        [[corners[25][0][0], corners[34][0][1]], [corners[34][0][0], corners[11][3][1]]], 
        [[corners[42][1][0], corners[52][1][1]], [corners[40][0][0], corners[10][3][1]-30]]   
    ]

    i=1
    eventlist=[]
    letters = {1: "A", 2: "B", 3: "C", 4: "D", 5: "E"}
    classconv = { "combat": "Combat", "destroyedbuilding": "Destroyed buildings", 
                 "humanitarianaid": "Humanitarian Aid and rehabilitation",
                 "militaryvehicles": "Military Vehicles", "fire": "Fire", "blank": "Blank"}

    device = "cuda" if torch.cuda.is_available() else "cpu"




    # model.classifier = torch.nn.Sequential(
    #     nn.Linear(1280, 256), # Additional linear layer with 256 output features
    #     # nn.ReLU(inplace=True),         # Activation function
    #     nn.Dropout(p=0.5, inplace=True),
    #     nn.Linear(256, 5)
    # ).to(device)
    # model.load_state_dict(torch.load('weights/w1.tf'))


    
    image_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224, 224), antialias=False),
            
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    temp = 'output/temp.jpg'

    for i, (tl, br) in enumerate(events):
        tl_adj = [tl[0] + 10, tl[1] + 7]
        br_adj = [br[0] - 10, br[1] - 4]
        roi = img[tl_adj[1]:br_adj[1], tl_adj[0]:br_adj[0]]

        reference = cv2.imread(f"images/empty/{i}.jpg")
        
        # Perform morphological opening
        kernel = np.ones((5,5),np.uint8)
        opened = cv2.morphologyEx(roi, cv2.MORPH_OPEN, kernel)

        gray = cv2.cvtColor(opened, cv2.COLOR_BGR2GRAY)

        # Apply adaptive thresholding to the image
        binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

        # Find contours in the image
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Sort contours by area in descending order, take the first one (the largest)
        contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

        # Get the bounding rectangle of the largest contour
        x, y, w, h = cv2.boundingRect(contour)

        # Crop the image using the bounding rectangle, add some padding if needed
        padding = 0  # adjust this value according to your needs
        crop = roi[max(0, y-padding):min(y+h+padding, roi.shape[0]), max(0, x-padding):min(x+w+padding, roi.shape[1])]

        # Convert the cropped image to HSV color space
        # hsv_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)

        # img_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        # lower_green = np.array([0, 50, 0])
        # upper_green = np.array([50, 100, 50])

        img_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        lower_green = np.array([200, 200, 200])
        upper_green = np.array([255, 255, 255])

        # Threshold the RGB image to get only green colors
        mask = cv2.inRange(img_rgb, lower_green, upper_green)

        # Calculate the percentage of green pixels
        green_percentage = (np.count_nonzero(mask) / (crop.shape[0] * crop.shape[1])) * 100
        # print(green_percentage)

        # If the majority of the cropped area is green, skip the classification
        threshold_percentage = 80  # Adjust the threshold as needed
        cv2.imwrite(temp, crop, [cv2.IMWRITE_JPEG_QUALITY, 100])
        crop = cv2.imread(temp)
        # Preprocess the image for TensorFlow
        tf_img = tf_image.load_img(temp, target_size=(224, 224))
        img_array = tf_image.img_to_array(tf_img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        # Make a prediction
        preds = model.predict(img_array)
        # preds = model.predict(crop)

        # Convert prediction probabilities to prediction labels
        pred = np.argmax(preds, axis=1)
        class_names = ['combat', 'destroyedbuilding', 'fire', 'humanitarianaid', 'militaryvehicles']


        print(pred[0])
        event = class_names[pred[0]]

        offset_x = tl_adj[0] + x - 10
        offset_y = tl_adj[1] + y - 10    

        box = cv2.rectangle(marking_img, (offset_x, offset_y), (offset_x + w + 20, offset_y + h + 20), (0, 255, 0), 2)
        
        offset_y -= 10
        font = cv2.FONT_HERSHEY_SIMPLEX
        scale = 0.7
        thickness = 1
        text = classconv[event]
        (text_width, text_height), _ = cv2.getTextSize(text, font, scale, thickness)

        cv2.rectangle(marking_img, (offset_x, offset_y - text_height - 10), (offset_x + text_width, offset_y), (140, 133, 133), -1)
        cv2.putText(box, text, (offset_x, offset_y - 10), cv2.FONT_HERSHEY_SIMPLEX, scale, (0,255,0), thickness)
        identified_labels[letters[i+1]] = classconv[event]
        # display_frame = cv2.resize(marking_img, (960, 540))
        cv2.imshow("Marked Image", marking_img)
        
        cv2.waitKey(500)  # delay for 500 milliseconds
        
    # display_frame = 
    cv2.imshow("Marked Image", marking_img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return identified_labels



    
    # ... (the rest of the function stays the same)

In [59]:
def empty(image_path, threshold):
    # global cap
    identified_labels = {}  
    
    # ret, frame = cap.read()
    # display_frame = cv2.resize(frame, (960, 540))

    # # Create a named window
    # cv2.namedWindow("Live Feed", cv2.WINDOW_NORMAL)

    # # Flag to check if the picture has been taken
    # picture_taken = False
    # # Get start time
    # start_time = time.time()
    # # Read and display frames from the camera

    # while not picture_taken:
    #     ret, frame = cap.read()
    #     display_frame = cv2.resize(frame, (960, 540))

    #     if not ret:
    #         print("Error reading frame from the camera")
    #         break

    #     cv2.imshow("Live Feed", display_frame)

    #     # Move the window to the left
    #     cv2.moveWindow("Live Feed", 0, 0)

    #     if time.time() - start_time >= 5:
    #         cv2.imwrite(image_path, frame, [cv2.IMWRITE_JPEG_QUALITY, 100])
    #         picture_taken = True

    #     if cv2.waitKey(1) & 0xFF == ord('q'):
    #         break


    # cv2.destroyAllWindows()
    img = cv2.imread(image_path)
    # os.remove(image_path)
    cv2.imshow("Marked", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    _, corners = detect_ArUco_details(img)
    # mark = mark_ArUco_image(img,  details, corners)

    marking_img = np.copy(img)
    _, corners = detect_ArUco_details(marking_img)
    
    events = [
        [[corners[25][3][0], corners[21][0][1]], [corners[21][0][0], corners[7][1][1]-10]],
        [[corners[31][1][0], corners[28][1][1]], [corners[30][0][0], corners[14][3][1]]],
        [corners[31][1], [corners[30][0][0], corners[11][3][1]]], 
        [[corners[25][0][0], corners[34][0][1]], [corners[34][0][0], corners[11][3][1]]], 
        [[corners[42][1][0], corners[52][1][1]], [corners[40][0][0], corners[10][3][1]-30]]   
    ]

    i=1
    eventlist=[]
    letters = {1: "A", 2: "B", 3: "C", 4: "D", 5: "E"}
    classconv = { "combat": "Combat", "destroyedbuilding": "Destroyed buildings", 
                 "humanitarianaid": "Humanitarian Aid and rehabilitation",
                 "militaryvehicles": "Military Vehicles", "fire": "Fire", "blank": "Blank"}

    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = efficientnet_v2_s().to(device)
    
    model.classifier = torch.nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features=1280, out_features=5, bias=True),
    ).to(device)


    # model.classifier = torch.nn.Sequential(
    #     nn.Linear(1280, 256), # Additional linear layer with 256 output features
    #     # nn.ReLU(inplace=True),         # Activation function
    #     nn.Dropout(p=0.5, inplace=True),
    #     nn.Linear(256, 5)
    # ).to(device)

    model.load_state_dict(torch.load('weights/weights.tf'))
    # model.load_state_dict(torch.load('weights/w1.tf'))


    model.eval()
    
    image_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224, 224), antialias=False),
            
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    temp = 'output/temp.jpg'

    empties = list(input("Empties : "))

    for i, (tl, br) in enumerate(events):
        tl_adj = [tl[0] + 10, tl[1] + 7]
        br_adj = [br[0] - 10, br[1] - 4]
        roi = img[tl_adj[1]:br_adj[1], tl_adj[0]:br_adj[0]]

        reference = cv2.imread(f"images/empty/{i}.jpg")
        
        # Perform morphological opening
        kernel = np.ones((5,5),np.uint8)
        opened = cv2.morphologyEx(roi, cv2.MORPH_OPEN, kernel)

        gray = cv2.cvtColor(opened, cv2.COLOR_BGR2GRAY)

        # Apply adaptive thresholding to the image
        binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

        # Find contours in the image
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Sort contours by area in descending order, take the first one (the largest)
        contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

        # Get the bounding rectangle of the largest contour
        x, y, w, h = cv2.boundingRect(contour)

        # Crop the image using the bounding rectangle, add some padding if needed
        padding = 0  # adjust this value according to your needs
        crop = roi[max(0, y-padding):min(y+h+padding, roi.shape[0]), max(0, x-padding):min(x+w+padding, roi.shape[1])]

        if letters[i+1] in empties:
            filename = f"images/emptyds/image_{int(time.time())}.jpg"
            cv2.imwrite(filename, crop)

        # Convert the cropped image to HSV color space
        # hsv_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)

        # img_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        # lower_green = np.array([0, 50, 0])
        # upper_green = np.array([50, 100, 50])

        img_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        lower_green = np.array([200, 200, 200])
        upper_green = np.array([255, 255, 255])

        # Threshold the RGB image to get only green colors
        mask = cv2.inRange(img_rgb, lower_green, upper_green)

        # Calculate the percentage of green pixels
        green_percentage = (np.count_nonzero(mask) / (crop.shape[0] * crop.shape[1])) * 100
        # print(green_percentage)

        # If the majority of the cropped area is green, skip the classification
        threshold_percentage = 80  # Adjust the threshold as needed
        cv2.imwrite(temp, crop, [cv2.IMWRITE_JPEG_QUALITY, 100])
        crop = cv2.imread(temp)




        eventlist.append(crop)
        cv2.imwrite(temp, crop, [cv2.IMWRITE_JPEG_QUALITY, 100])
        result = cv2.imread(temp, cv2.IMREAD_COLOR)
        result = super_resolution.cartoon_upsampling_4x(temp, temp )


        with torch.inference_mode():
            # 6. Transform and add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
            transformed_image = image_transform(result).unsqueeze(dim=0)
            # 7. Make a prediction on image with an extra dimension and send it to the target device
            target_image_pred = model(transformed_image.to(device))

        # 8. Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
        target_image_pred_probs = torch.softmax(target_image_pred, dim=1)
        
        # 9. Convert prediction probabilities -> prediction labels
        pred = torch.argmax(target_image_pred_probs, dim=1)
        class_names = ['combat', 'destroyedbuilding', 'fire', 'humanitarianaid', 'militaryvehicles']

        print(target_image_pred_probs[0])

        if max(target_image_pred_probs[0]) < threshold[i]:
            event = "blank"
        else:
            
            event = class_names[pred]

        offset_x = tl_adj[0] + x - 10
        offset_y = tl_adj[1] + y - 10    

        box = cv2.rectangle(marking_img, (offset_x, offset_y), (offset_x + w + 20, offset_y + h + 20), (0, 255, 0), 2)
        
        offset_y -= 10
        font = cv2.FONT_HERSHEY_SIMPLEX
        scale = 0.7
        thickness = 1
        text = classconv[event]
        (text_width, text_height), _ = cv2.getTextSize(text, font, scale, thickness)

        cv2.rectangle(marking_img, (offset_x, offset_y - text_height - 10), (offset_x + text_width, offset_y), (140, 133, 133), -1)
        cv2.putText(box, text, (offset_x, offset_y - 10), cv2.FONT_HERSHEY_SIMPLEX, scale, (0,255,0), thickness)
        identified_labels[letters[i+1]] = classconv[event]
        # display_frame = cv2.resize(marking_img, (960, 540))
        cv2.imshow("Marked Image", marking_img)
        
        cv2.waitKey(500)  # delay for 500 milliseconds
        
    # display_frame = 
    cv2.imshow("Marked Image", marking_img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return identified_labels


In [60]:
def mark_ArUco_image(image,ArUco_details_dict, ArUco_corners):

    for ids, details in ArUco_details_dict.items():
        center = details[0]
        cv2.circle(image, center, 5, (0,0,255), -1)

        corner = ArUco_corners[int(ids)]
        cv2.circle(image, (int(corner[0][0]), int(corner[0][1])), 5, (50, 50, 50), -1)
        cv2.circle(image, (int(corner[1][0]), int(corner[1][1])), 5, (0, 255, 0), -1)
        cv2.circle(image, (int(corner[2][0]), int(corner[2][1])), 5, (128, 0, 255), -1)
        cv2.circle(image, (int(corner[3][0]), int(corner[3][1])), 5, (25, 255, 255), -1)

        tl_tr_center_x = int((corner[0][0] + corner[1][0]) / 2)
        tl_tr_center_y = int((corner[0][1] + corner[1][1]) / 2) 

        cv2.line(image,center,(tl_tr_center_x, tl_tr_center_y),(255,0,0),5)
        display_offset = int(math.sqrt((tl_tr_center_x - center[0])**2+(tl_tr_center_y - center[1])**2))
        cv2.putText(image,str(ids),(center[0]+int(display_offset/2),center[1]),cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        angle = details[1]
        cv2.putText(image,str(angle),(center[0]-display_offset,center[1]),cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
    return image

In [61]:
tf.__version__

'2.15.0'

In [62]:
# img = cv2.imread("images/all4.jpg")
# details, corners = detect_ArUco_details(img)
# marked = mark_ArUco_image(img, details, corners)
# cv2.imshow("Marked", marked)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [63]:
files = os.listdir("images/new")
for file in files:
    new_file = str("images/new/" + file)
    try:
        task_4a_returnkeras(new_file, [0]*5)
        # task_4a_return(new_file, [0]*5)
    except IndexError:
        continue
    except KeyError:
        continue


ValueError: Unknown object: 'Functional'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
# task_4a_return("images/captured.jpg", [0]*5)